# Yanchep Lagoon Model

First we need to load the IPython libraries:

In [ ]:
%pylab inline
%matplotlib inline
import os
import glob
import h5py
import netCDF4
import numpy as np
from scipy import signal
from scipy import interpolate

# Running XBeach

To run **XBeach**, we need to go into the folder containing the input file `params.txt`. 

In this first example this is the folder `default`. We use the `!` to tell IPython that we want to execute the code in `bash` mode. Similar to running it from a linux terminal.

In [ ]:
!pwd

In [ ]:
!xbeach

# Loading XBeach output

The netcdf file is located in the same location as your input files, we use the IPython netCDF4 functionality to load the data file. Using the `variables.keys()`, we can see all the parameters outputed by XBeach

In [ ]:
nc_data = netCDF4.Dataset('xboutput.nc')
print nc_data.variables.keys()

To query the type/definition of each of the `netcdf` keys we can do:

In [ ]:
nc_data.variables['globalx']

# Get visualisation parameters

In [ ]:
# Export the stored time values
time = nc_data.variables['globaltime'][:]
# Export the X-axis values of the profile
x = nc_data.variables['globalx'][:,:]
# Export the X-axis values of the profile
y = nc_data.variables['globaly'][:,:]
# Export the bed elevation
zb = nc_data.variables['zb'][:,:,:]
# Export the wave height
H = nc_data.variables['H'][:,:,:]
# Export the water level
zs = nc_data.variables['zs'][:,:,:]
# Export sedimentation/erosion
sedero = nc_data.variables['sedero'][:,:,:]
# Export total bed level change due to avalanching
dzav = nc_data.variables['dzav'][:,:,:]
# Export x-axis velocity
u = nc_data.variables['u'][:,:,:]
# Export y-axis velocity
v = nc_data.variables['v'][:,:,:]
# Export x-axis wave force
Fx = nc_data.variables['Fx'][:,:,:]
# Export y-axis wave force
Fy = nc_data.variables['Fy'][:,:,:]

Find the grid resolution

In [ ]:
diffx = np.diff(x,axis=1)
diffy = np.diff(y,axis=1)
dx = np.sqrt(diffx.min()**2+diffy.min()**2)
print 'Model resolution (m):',dx

## Interpolation on a 5-m regular grid

First we define the 5-m resolution coordinate system based on Yanchep grid:

In [ ]:
nx = x.shape[1]
ny = x.shape[0]
xi = np.linspace(0., nx*dx, nx)
yi = np.linspace(0., ny*dx, ny)
mxi, myi = np.meshgrid(xi, yi)

# We define the new coordinate system
xnew = np.linspace(0., nx*dx, 2*nx+1)
ynew = np.linspace(0., ny*dx, 2*ny+1)
newxi, newyi = np.meshgrid(xnew, ynew)

In [ ]:
def gaussianFilter(sizex,sizey=None,scale=0.333):
    '''
    Generate and return a 2D Gaussian function
    of dimensions (sizex,sizey)

    If sizey is not set, it defaults to sizex
    A scale can be defined to widen the function (default = 0.333)
    '''
    sizey = sizey or sizex
    x, y = np.mgrid[-sizex:sizex+1, -sizey:sizey+1]
    g = np.exp(-scale*(x**2/float(sizex)+y**2/float(sizey)))
    
    return g/g.sum()

def smoothData(dem, smth=1):
    '''
    Calculate the slope and gradient of a DEM
    '''
    
    gaussZ = np.zeros((dem.shape[0]+6,dem.shape[1]+6))
    gaussZ[3:-3,3:-3] = dem

    f0 = gaussianFilter(smth)
    smoothDEM = signal.convolve(gaussZ,f0,mode='valid')
    
    return smoothDEM[2:-2,2:-2]

## Create the hdf5 files for Paraview

First we set the folder name that will be used to store the file

In [ ]:
outDir = 'output'

if os.path.exists(outDir):
    outDir += '_'+str(len(glob.glob(outDir+str('*')))-1)
os.makedirs(outDir)

print 'data will be stored in the following folder: ',outDir

Then we write the files

In [ ]:
for step in range(len(time)-1):
    print 'Performing step:',step,'out of:',len(time)-1
    f1 = interpolate.interp2d(xi, yi, zb[step,:,:], kind='cubic')
    znew = f1(xnew, ynew)
    f2 = interpolate.interp2d(xi, yi, H[step,:,:]+zs[step,:,:], kind='cubic')
    Hnew = f2(xnew, ynew)
    f3 = interpolate.interp2d(xi, yi, sedero[step,:,:], kind='cubic')
    sederonew = f3(xnew, ynew)
    f4 = interpolate.interp2d(xi, yi, dzav[step,:,:], kind='cubic')
    dzavnew = f4(xnew, ynew)
    f5 = interpolate.interp2d(xi, yi, u[step,:,:], kind='cubic')
    unew = f5(xnew, ynew)
    f6 = interpolate.interp2d(xi, yi, v[step,:,:], kind='cubic')
    vnew = f6(xnew, ynew)
    f7 = interpolate.interp2d(xi, yi, Fx[step,:,:], kind='cubic')
    Fxnew = f7(xnew, ynew)
    f8 = interpolate.interp2d(xi, yi, Fy[step,:,:], kind='cubic')
    Fynew = f8(xnew, ynew)
    
    h5file = str(outDir)+'/data'+str(step)+'.hdf5'
    h5ff = 'data'+str(step)+'.hdf5'
    with h5py.File(h5file, "w") as f:
        f.create_dataset('zb',shape=(znew.shape), dtype='float32', compression='gzip')
        f["zb"][:,:] = znew
        f.create_dataset('H',shape=(Hnew.shape), dtype='float32', compression='gzip')
        f["H"][:,:] = Hnew
        f.create_dataset('sedero',shape=(sederonew.shape), dtype='float32', compression='gzip')
        f["sedero"][:,:] = sederonew
        f.create_dataset('dzav',shape=(dzavnew.shape), dtype='float32', compression='gzip')
        f["dzav"][:,:] = dzavnew
        f.create_dataset('u',shape=(unew.shape), dtype='float32', compression='gzip')
        f["u"][:,:] = unew
        f.create_dataset('v',shape=(vnew.shape), dtype='float32', compression='gzip')
        f["v"][:,:] = vnew
        f.create_dataset('Fx',shape=(Fxnew.shape), dtype='float32', compression='gzip')
        f["Fx"][:,:] = Fxnew
        f.create_dataset('Fy',shape=(Fynew.shape), dtype='float32', compression='gzip')
        f["Fy"][:,:] = Fynew
        
    xmf_file = str(outDir)+'/bed'+str(step)+'.xmf'
    f= open(str(xmf_file),'w')
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write('<!DOCTYPE Xdmf SYSTEM "Xdmf.dtd">\n')
    f.write('<Xdmf Version="2.0" xmlns:xi="http://www.w3.org/2001/XInclude">\n')
    f.write(' <Domain>\n')
    f.write('    <Grid GridType="Collection" CollectionType="Spatial">\n')
    f.write('      <Time Type="Single" Value="%s"/>\n'%time[step])
    f.write('      <Grid Name="Block.0" GridType="Uniform">\n')
    f.write('         <Topology Type="2DCoRectMesh" NumberOfElements=')
    f.write('"%d %d"/>\n'%(znew.shape[0],znew.shape[1]))
    f.write('         <Geometry Type="ORIGIN_DXDY">\n')
    f.write('          <DataItem NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d">0. 0.</DataItem>\n'%2)
    f.write('          <DataItem NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d">10. 10.</DataItem>\n'%2)
    f.write('         </Geometry>\n')

    f.write('         <Attribute Type="Scalar" Center="Node" Name="zb [m]">\n')
    f.write('          <DataItem Format="HDF" NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d %d">%s:/zb</DataItem>\n'%(len(ynew),len(xnew),h5ff))
    f.write('         </Attribute>\n')

    f.write('         <Attribute Type="Scalar" Center="Node" Name="sedero [m]">\n')
    f.write('          <DataItem Format="HDF" NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d %d">%s:/sedero</DataItem>\n'%(len(ynew),len(xnew),h5ff))
    f.write('         </Attribute>\n')
    
    f.write('         <Attribute Type="Scalar" Center="Node" Name="dzav [m]">\n')
    f.write('          <DataItem Format="HDF" NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d %d">%s:/dzav</DataItem>\n'%(len(ynew),len(xnew),h5ff))
    f.write('         </Attribute>\n')
    
    f.write('      </Grid>\n')
    f.write('    </Grid>\n')
    f.write(' </Domain>\n')
    f.write('</Xdmf>\n')
    f.close()
    
    
    xmf_file = str(outDir)+'/sea'+str(step)+'.xmf'
    f= open(str(xmf_file),'w')
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write('<!DOCTYPE Xdmf SYSTEM "Xdmf.dtd">\n')
    f.write('<Xdmf Version="2.0" xmlns:xi="http://www.w3.org/2001/XInclude">\n')
    f.write(' <Domain>\n')
    f.write('    <Grid GridType="Collection" CollectionType="Spatial">\n')
    f.write('      <Time Type="Single" Value="%s"/>\n'%time[step])
    f.write('      <Grid Name="Block.0" GridType="Uniform">\n')
    f.write('         <Topology Type="2DCoRectMesh" NumberOfElements=')
    f.write('"%d %d"/>\n'%(znew.shape[0],znew.shape[1]))
    f.write('         <Geometry Type="ORIGIN_DXDY">\n')
    f.write('          <DataItem NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d">0. 0.</DataItem>\n'%2)
    f.write('          <DataItem NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d">10. 10.</DataItem>\n'%2)
    f.write('         </Geometry>\n')

    f.write('         <Attribute Type="Scalar" Center="Node" Name="H+zs [m]">\n')
    f.write('          <DataItem Format="HDF" NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d %d">%s:/H</DataItem>\n'%(len(ynew),len(xnew),h5ff))
    f.write('         </Attribute>\n')
    
    f.write('         <Attribute Type="Scalar" Center="Node" Name="u [m/s]">\n')
    f.write('          <DataItem Format="HDF" NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d %d">%s:/u</DataItem>\n'%(len(ynew),len(xnew),h5ff))
    f.write('         </Attribute>\n')
    
    f.write('         <Attribute Type="Scalar" Center="Node" Name="v [m/s]">\n')
    f.write('          <DataItem Format="HDF" NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d %d">%s:/v</DataItem>\n'%(len(ynew),len(xnew),h5ff))
    f.write('         </Attribute>\n')
    
    f.write('         <Attribute Type="Scalar" Center="Node" Name="Fx [N/m2]">\n')
    f.write('          <DataItem Format="HDF" NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d %d">%s:/Fx</DataItem>\n'%(len(ynew),len(xnew),h5ff))
    f.write('         </Attribute>\n')
    
    f.write('         <Attribute Type="Scalar" Center="Node" Name="Fy [N/m2]">\n')
    f.write('          <DataItem Format="HDF" NumberType="Float" Precision="4" ')
    f.write('Dimensions="%d %d">%s:/Fy</DataItem>\n'%(len(ynew),len(xnew),h5ff))
    f.write('         </Attribute>\n')
    
    f.write('      </Grid>\n')
    f.write('    </Grid>\n')
    f.write(' </Domain>\n')
    f.write('</Xdmf>\n')
    f.close()
    
xdmf_file = str(outDir)+'/bedSeries.xdmf'
f= open(str(xdmf_file),'w')

f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
f.write('<!DOCTYPE Xdmf SYSTEM "Xdmf.dtd">\n')
f.write('<Xdmf Version="2.0" xmlns:xi="http://www.w3.org/2001/XInclude">\n')
f.write(' <Domain>\n')
f.write('    <Grid GridType="Collection" CollectionType="Temporal">\n')

for step in range(len(time)-1):
    xfile = 'bed'+str(step)+'.xmf'
    f.write('      <xi:include href="%s" xpointer="xpointer(//Xdmf/Domain/Grid)"/>\n' %xfile)

f.write('    </Grid>\n')
f.write(' </Domain>\n')
f.write('</Xdmf>\n')
f.close()

xdmf_file = str(outDir)+'/seaSeries.xdmf'
f= open(str(xdmf_file),'w')

f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
f.write('<!DOCTYPE Xdmf SYSTEM "Xdmf.dtd">\n')
f.write('<Xdmf Version="2.0" xmlns:xi="http://www.w3.org/2001/XInclude">\n')
f.write(' <Domain>\n')
f.write('    <Grid GridType="Collection" CollectionType="Temporal">\n')

for step in range(len(time)-1):
    xfile = 'sea'+str(step)+'.xmf'
    f.write('      <xi:include href="%s" xpointer="xpointer(//Xdmf/Domain/Grid)"/>\n' %xfile)

f.write('    </Grid>\n')
f.write(' </Domain>\n')
f.write('</Xdmf>\n')
f.close()

To visualise the output in Paraview we will need to copy the folder from the container to the local repository (attached volume).

This is done using the following command:

In [ ]:
!cp -rf output /workspace/volume/